In [118]:
import pandas as pd

data = pd.read_csv('./Dataset1/football_data.csv')
data.head()

,Unnamed: 0,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,...,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Unnamed: 60
0,0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,...,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M,NaN
1,1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,...,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M,NaN
2,2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,...,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M,NaN
3,3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,...,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M,NaN
4,4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,...,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M,NaN


In [119]:
data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Nationality', 'Overall',
       'Potential', 'Club', 'Value', 'Wage', 'Special', 'Preferred Foot',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate',
       'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined',
       'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Release Clause',
       'Unnamed: 60'],
      dtype='object')

In [120]:
data.drop(columns=['Unnamed: 0', 'ID', 'Name', 'Loaned From', 'Unnamed: 60'], inplace=True)

In [121]:
data['Contract Valid Until'] = [x[-4: ] if len(str(x)) > 4 else x for x in data['Contract Valid Until']]

In [122]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

data['Nationality'] = encoder.fit_transform(data['Nationality'])
data['Club'] = encoder.fit_transform(data['Club'])
data['Value'] = encoder.fit_transform(data['Value'])
data['Wage'] = encoder.fit_transform(data['Wage'])
data['Preferred Foot'] = encoder.fit_transform(data['Preferred Foot'])
data['Work Rate'] = encoder.fit_transform(data['Work Rate'])
data['Body Type'] = encoder.fit_transform(data['Body Type'])
data['Position'] = encoder.fit_transform(data['Position'])
data['Real Face'] = encoder.fit_transform(data['Real Face'])
data['Joined'] = encoder.fit_transform(data['Joined'])
data['Weight'] = encoder.fit_transform(data['Weight'])
data['Height'] = encoder.fit_transform(data['Height'])
data['Release Clause'] = encoder.fit_transform(data['Release Clause'])


In [123]:
for index, row in data.iterrows():
    try:
        data.loc[index] = row.astype('float')
    except ValueError as e:
        data = data.drop(index, axis=0)

data = data.reset_index(drop=True)

18179
